# TCC Feedback and "Feedforward" Alternated front back, Filling Comparison
**Separate Variable**

For Hybrid fill, GRU threshold is 4, which means equal or more than 4 consecutive blank, GRU is used

## Function

In [ ]:
import os
import pathlib

import numpy as np
import scipy as sp
import scipy.stats as sp_stats
import scipy.interpolate as sp_interp
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import LZH_Utilities as utl

In [ ]:
USING_STANDARD_DIVIATION = True

In [ ]:
DATA_FILE_PATH = "full_rank_dataset_anormaly"


OUTPUT_FOLDER = pathlib.Path("Output/filling_comp_separate_vars_sd_metric") if USING_STANDARD_DIVIATION else pathlib.Path("Output/filling_comp_separate_vars")


RANDOM_DATA_FOLDER_PATH = OUTPUT_FOLDER / "filling_comparison/"
os.makedirs(RANDOM_DATA_FOLDER_PATH, exist_ok=True)
RANDOM_DATA_FOLDER_ARCHIVE = utl.Archive(RANDOM_DATA_FOLDER_PATH)

REALISTIC_DATA_OUTPUT_FOLDER_PATH = OUTPUT_FOLDER / "filling_comparison_realistic/"
os.makedirs(REALISTIC_DATA_OUTPUT_FOLDER_PATH, exist_ok=True)
REALISTIC_DATA_OUTPUT_FOLDER_PATH_ARCHIVE = utl.Archive(REALISTIC_DATA_OUTPUT_FOLDER_PATH)

In [ ]:
def work():
    def save_all_data(archive:utl.Archive, time_stamp, fill_MSE, fill_MAE):
        archive.save_data(fill_MAE, "FILL_MAE_{0}.pkl".format(time_stamp))
        archive.save_data(fill_MSE, "FILL_MSE_{0}.pkl".format(time_stamp))

    def load_data(file_name):
        """
        return as dictionary that matches
        """
        ret_val = {}
        df = pd.read_csv(file_name)
        for col_name in df.columns:
            ret_val[col_name] = df[col_name].to_numpy()
        return ret_val

    def transpose_stack(arr):
        return np.array([arr]).T

    def tsr(arr):
        return torch.tensor(arr)

    def plot(x, y, x_label="", y_label="", legend="", title=""):
        # plt.figure(figsize=[8, 6], dpi=300)
        plt.figure(figsize=[8, 6])

        if (type(legend) is list):
            for yy in y:
                plt.plot(x, yy)
            plt.legend(legend)
        else: 
            plt.plot(x, y)

        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.title(title)
        plt.show()

    def plot_diff_percentage(x, y, x_label="", y_label="", title=""):
        # plt.figure(figsize=[8, 6], dpi=300)
        plt.figure(figsize=[8, 6])

        x_fit = np.linspace(0, 100, 1000)
        y_fit1 = np.polyval(np.polyfit(x, y, 1), x_fit)

        plt.scatter(x, y, s=0.5, c='k')
        plt.plot(x_fit, y_fit1, "r")

        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.legend(["linear regression", "data"])
        plt.ylim([0, 100])
        plt.xlim([0, 100])
        plt.title(title)
        plt.show()

        result = sp_stats.linregress(x, y)
        print("     slope: {0}".format(result.slope))
        print(" intercept: {0}".format(result.intercept))
        print("corr coeff: {0}".format(result.rvalue))
        print("  variance: {0}".format(result.rvalue ** 2))


    def plot_diff_percentage_bined_average(x, y, x_label="", y_label="", title="", bin_count=10):
        # plt.figure(figsize=[8, 6], dpi=300)
        plt.figure(figsize=[8, 6])

        # Overall
        x_fit = np.linspace(0, 100, 10)
        y_fit1 = np.polyval(np.polyfit(x, y, 1), x_fit)

        plt.scatter(x, y, s=0.5, alpha=0.3, c='k', label="data")
        plt.plot(x_fit, y_fit1, "r", label="linear regression")

        plt.xlabel(x_label)
        plt.ylabel(y_label)

        result = sp_stats.linregress(x, y)
        print("Overall:")
        print("     slope: {0}".format(result.slope))
        print(" intercept: {0}".format(result.intercept))
        print("corr coeff: {0}".format(result.rvalue))
        print("  variance: {0}".format(result.rvalue ** 2))
        print()

        # Binned Average
        is_label_printed = False

        bin_arr = np.linspace(0, 100, bin_count+1)
        for (low, high) in zip(bin_arr[:-1], bin_arr[1:]):

            bin_filter = np.logical_and(low <= x, x <= high)
            x_bin = x[bin_filter]
            y_bin = y[bin_filter]

            x_fit = np.linspace(low, high, 10)
            y_fit1 = np.polyval(np.polyfit(x_bin, y_bin, 1), x_fit)

            if (not is_label_printed):
                plt.plot(x_fit, y_fit1, "b", label="binned linear reg.")
                is_label_printed = True
            else:
                plt.plot(x_fit, y_fit1, "b")

            result = sp_stats.linregress(x_bin, y_bin)
            print("[{4:>3}, {5:>3}):   b:{0:.4f};  m:{1:.4f}; r:{2:.4f}; var:{3:.4f}".format(result.slope, result.intercept, result.rvalue, result.rvalue ** 2, int(low), int(high)))


        plt.legend()
        plt.ylim([0, 100])
        plt.xlim([0, 100])
        plt.title(title)
        plt.show()

    def flatten_time_axis(data):
        """
        flatten the time axis so it could be used in FC and linear model. 
        """
        return np.concatenate(data, axis=0)

    ## Data

    df = utl.read_time_series_data("full_rank_dataset")

    for col_name in ['LTS', 'SST', 'Subsidence', 'Night_Day', 'RH', 'q', 'wsp', 'TCC']:
        df[-1][col_name + "_FF"] = 0
        for idx in np.arange(8):
            df[idx][col_name + "_FF"] = df[idx + 1][col_name]

    df[0]["TCC_FB"] = 0
    for idx in np.arange(8):
        df[idx + 1]["TCC_FB"] = df[idx]["TCC"]

    input_col = ['LTS', 'SST', 'Subsidence', 'Night_Day', 'RH', 'q', 'wsp', "TCC_FB"]
    output_col = ['LTS_FF', 'SST_FF', 'Subsidence_FF', 'Night_Day_FF', 'RH_FF', 'q_FF', 'wsp_FF', "TCC"]

    idx_test_set = np.random.choice(np.arange(df[0].shape[0]), [int(0.1 * df[0].shape[0])], False)
    temp = np.delete(np.arange(df[0].shape[0]), idx_test_set)
    idx_final_test_set = np.random.choice(temp.shape[0], [int(0.1 * df[0].shape[0])], False)
    idx_training_set = np.delete(temp, idx_final_test_set)

    # print("Training: {0}; Testing: {1}; FinalTesting: {2}".format(idx_training_set.shape[0], idx_test_set.shape[0], idx_final_test_set.shape[0]))

    time_arr = np.arange(9)

    X_full = [df[time][input_col].to_numpy() for time in time_arr]
    y_hat_full = [np.c_[df[time][output_col].to_numpy()] for time in time_arr]

    X_train = np.array([X_full[time][idx_training_set] for time in time_arr])
    y_hat_train = np.array([y_hat_full[time][idx_training_set] for time in time_arr])

    X_test = np.array([X_full[time][idx_test_set] for time in time_arr])
    y_hat_test = np.array([y_hat_full[time][idx_test_set] for time in time_arr])

    X_final_test = np.array([X_full[time][idx_final_test_set] for time in time_arr])
    y_hat_final_test = np.array([y_hat_full[time][idx_final_test_set] for time in time_arr])

    X_train_RRR     = X_train     [::-1].copy()
    y_hat_train_RRR = y_hat_train [::-1].copy()

    # The following is complicated, we are basically back in time. Pls draw the graph and confirm this is correct. 
    X_test_RRR      = y_hat_test  [::-1].copy()
    y_hat_test_RRR  = X_test      [::-1].copy()

    X_final_test_RRR     = y_hat_final_test  [::-1].copy()
    y_hat_final_test_RRR = X_final_test      [::-1].copy()

    ## Filling Model

    class MLTCC_Model_GRU_Normalized:
        def __init__(self, layer_overwrite=None):
            self.NMR = utl.Normalizer()

            self.input_size = 8
            self.output_size = 8
            self.batch_size = 10000
            self.step_size = 1e-3

            h_gru = 50
            h1 = 20
            h2 = 40
            h3 = 20

            if (layer_overwrite is not None):
                (h_gru, h1, h2, h3) = layer_overwrite

            self.gru = nn.GRU(self.input_size, h_gru, 2, batch_first=False)   # (seq, batch, feature)
            self.fc1 = nn.Linear(h_gru, h1)
            self.fc2 = nn.Linear(h1, h2)
            self.fc3 = nn.Linear(h2, h3)
            self.fc4 = nn.Linear(h3, self.output_size)

            variable_list = [
                    self.fc1.weight, self.fc1.bias,
                    self.fc2.weight, self.fc2.bias,
                    self.fc3.weight, self.fc3.bias,
                    self.fc4.weight, self.fc4.bias
                ]
            for all_weights in self.gru.all_weights:
                for weights in all_weights:
                    variable_list.append(weights)


            self.optim = torch.optim.Adam(
                variable_list,
                lr=self.step_size
            )

            # Step function
            self.step_h_last = None
            self.step_y_hat_last = None
            self.step_last_DayNight = None

        def foward(self, X):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
            y := [TCC],                  (sample_size, 1) like tensor
            """
    #         seq_X_result = []
    #         for seq_X in X:
    #             seq_X_result.append(torch.relu(self.fc1(seq_X.float())))
    #         seq_X_result = tsr(seq_X_result)
            # normalize
            newX = []
            for idx in np.arange(X.shape[0]):
                newX.append(self.NMR.normalize_input(X[idx]).detach().numpy())
            X = torch.tensor(np.array(newX))

            X = X.float()

            X_gru_outputs, hn = self.gru(X)
            X = torch.relu(X_gru_outputs[-1]) # use last output

            X = torch.relu(self.fc1(X))
            X = torch.relu(self.fc2(X))
            X = torch.relu(self.fc3(X))
            X = self.fc4(X)

            return self.NMR.denormalize_output(X)

        def step_forward(self, X_step=None):
            """
            X_step is in numpy.array in (n, 8)
            reutrn value is in numpy.array in (n, 8)

            if X_step is None, the algorithm will use the previous result as input
            Any np.nan value in X_step will be replaced by corrisponding previous result. 
            """
            output_tensor = None
            if (X_step is None):
                if (self.step_h_last is None):
                    # we do not have previous result
                    return None
                else:
                    # specify the implied DayNight data
                    # self.step_last_DayNight = 1 - self.step_last_DayNight # flip DayNight
                    self.step_last_DayNight = np.where(self.step_last_DayNight > 0.5, 0, 1)
                    self.step_y_hat_last[:, 3] = self.step_last_DayNight 

                    input_tensor = torch.tensor(np.array([self.NMR.normalize_input_nparray(self.step_y_hat_last)])).float()
                    output_tensor, self.step_h_last = self.gru(input_tensor, self.step_h_last)
            else:            
                if (self.step_h_last is None):
                    # we do not have previous result
                    input_tensor = torch.tensor(np.array([self.NMR.normalize_input_nparray(X_step)])).float()
                    output_tensor, self.step_h_last = self.gru(input_tensor)
                else:
                    for i0 in np.arange(X_step.shape[0]):
                        for i1 in np.arange(X_step.shape[1]):
                            if (np.isnan(X_step[i0, i1])):
                                X_step[i0, i1] = self.step_y_hat_last[i0, i1]
                    input_tensor = torch.tensor(np.array([self.NMR.normalize_input_nparray(X_step)])).float()
                    output_tensor, self.step_h_last = self.gru(input_tensor, self.step_h_last)

                # record last DayNight data
                self.step_last_DayNight = X_step[:, 3]

            X = torch.relu(output_tensor[-1]) # use last output

            X = torch.relu(self.fc1(X))
            X = torch.relu(self.fc2(X))
            X = torch.relu(self.fc3(X))
            X = self.fc4(X)

            self.step_y_hat_last = self.NMR.denormalize_output(X).detach().numpy()

            return self.step_y_hat_last

        def step_reset(self):
            self.step_h_last = None
            self.step_y_hat_last = None
            return

        def loss(self, y, y_hat):
            """
            y := [TCC],                  (sample_size, 1) like tensor
            y_hat := [TCC],              (sample_size, 1) like tensor
            """
            return F.mse_loss(
                self.NMR.normalize_input(y).float(), 
                self.NMR.normalize_input(y_hat).float()
            )

        def train(self, X, y_hat, max_epoch):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """
            sample_count = X.shape[1]

            epoch_num = []
            losses = []

            for i in np.arange(max_epoch):
                for idx_batch in np.arange(int(sample_count / self.batch_size))* self.batch_size:
                    X_batch = torch.tensor(X[:, idx_batch:idx_batch + self.batch_size, :])
                    y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                    y_batch = self.foward(X_batch)
                    loss = self.loss(y_batch, y_hat_batch)

                    self.optim.zero_grad()
                    loss.backward()
                    self.optim.step()

                idx_rand = np.random.randint(0, sample_count, [self.batch_size])
                X_batch = torch.tensor(X[:, idx_rand, :]).float()
                y_hat_batch = torch.tensor(y_hat[idx_rand, :]).float()
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)

                epoch_num.append(i)
                losses.append(loss.item())

            return (epoch_num, losses)


        def train_time_series(self, X, y_hat, max_epoch, max_epoch_per_time):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """

            progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")

            # setting normalizer
            # flatten X and y_hat
            X_flat = np.concatenate(X, axis=0)
            y_hat_flat = np.concatenate(y_hat, axis=0)

            self.NMR.set_mean_and_sd(torch.tensor(X_flat), torch.tensor(y_hat_flat))


            full_losses = np.array([])

            for i in np.arange(max_epoch):
                for time in np.arange(X.shape[0]):
                    (x_axis, losses) = self.train(X[:time+1, :, :], y_hat[time, :, :], max_epoch_per_time)
                    full_losses = np.concatenate((full_losses, losses))

                progress_bar.update(i + 1, "epoch {0}".format(i))

            progress_bar.update().end()

            return (np.arange(len(full_losses)) + 1, full_losses)

    # forward training data
    X_train_A = X_train[:-1].copy()
    y_hat_train_A = y_hat_train[:-1].copy()
    # [:-1] is due to feedforward process

    # backward traning data
    X_train_RRR_A = X_train_RRR[1:-1].copy()
    y_hat_train_RRR_A = y_hat_train_RRR[1:-1].copy()
    # [1:-1] is due to feedforward process

    ### Forward Training

    tcc_model_gru = MLTCC_Model_GRU_Normalized()

    (x_axis, losses) = tcc_model_gru.train_time_series(X_train_A, y_hat_train_A, 50, 2)

    ### Backward Training

    # tcc_model_gru_RRR = MLTCC_Model_GRU_Normalized(layer_overwrite=[10, 15, 10, 10])
    tcc_model_gru_RRR = MLTCC_Model_GRU_Normalized()

    (x_axis, losses) = tcc_model_gru_RRR.train_time_series(X_train_RRR_A, y_hat_train_RRR_A, 50, 2)

    ## Filling Function

    def idx_full_rank(nparr):
        return np.sum(np.sum(np.logical_not(np.isnan(nparr)), axis=2), axis=0) == 9 * 8
    def count_full_rank(nparr):
        return np.count_nonzero(idx_full_rank(nparr))

    def GRU_fill(nparr_fill, tcc_model_gru, tcc_model_gru_RRR, show_progress=True):
        nparr_fill = nparr_fill.copy()
        filling_switching_threshold = 6

        if (show_progress):
            print("Full Rank Data: {0}".format(count_full_rank(nparr_fill)))

        # ------- forward filling -------
        good_data_mask = (np.sum(np.logical_not(np.isnan(nparr_fill)), axis=2) >= 8)

        # undefined indices are set to negative
        first_good_idx = np.zeros(good_data_mask.shape[1]) - 1 # set to -1

        for i1 in np.arange(good_data_mask.shape[0]):
            mask = first_good_idx < 0
            first_good_idx[mask] = (good_data_mask[i1][mask] * 2 - 1) * (i1 + 1) - 1

        # filling
        nparr_fill_res = nparr_fill.copy()

        for starting_idx in np.arange(filling_switching_threshold):  # 0 -> filling_switching_threshold(excl.)
            mask = first_good_idx == starting_idx

            if (show_progress):
                progress_bar = utl.TimedProgressBar(8 - starting_idx).update_msg("{0} forward pass, total {1}".format(starting_idx, np.count_nonzero(mask))).update()

            tcc_model_gru.step_reset()
            for time in np.arange(starting_idx, 8):
                nparr_fill_res[time + 1, mask] = tcc_model_gru.step_forward(nparr_fill[time, mask])

                if (show_progress):
                    progress_bar.inc_val().update()
            if (show_progress):
                print()

        # update
        if (show_progress):
            print("Full Rank Data: {0} -> {1}".format(count_full_rank(nparr_fill), count_full_rank(nparr_fill_res)))
        update_mask = np.isnan(nparr_fill)
        nparr_fill[update_mask] = nparr_fill_res[update_mask]

        # ------- backward filling -------
        good_data_mask = (np.sum(np.logical_not(np.isnan(nparr_fill)), axis=2) >= 8)

        # undefined indices are set to negative
        first_good_idx = np.zeros(good_data_mask.shape[1]) - 1 # set to -1

        for i1 in np.arange(good_data_mask.shape[0])[::-1]:
            first_good_idx[first_good_idx < 0] = (good_data_mask[i1][first_good_idx < 0] * 2 - 1) * (i1 + 1) - 1

        # filling
        nparr_fill_res = nparr_fill.copy()

        for starting_idx in np.arange(filling_switching_threshold, 9):  # filling_switching_threshold(incl.) -> 8
            mask = first_good_idx == starting_idx

            if (show_progress):
                progress_bar = utl.TimedProgressBar(np.arange(starting_idx, 0, -1).shape[0]).update_msg("{0} backward pass, total {1}".format(starting_idx, np.count_nonzero(mask))).update()

            tcc_model_gru_RRR.step_reset()
            for time in np.arange(starting_idx, 0, -1):
                nparr_fill_res[time - 1, mask] = tcc_model_gru_RRR.step_forward(nparr_fill[time, mask])

                if (show_progress):
                    progress_bar.inc_val().update()
            if (show_progress):
                print()


        # update
        if (show_progress):
            print("Full Rank Data: {0} -> {1}".format(count_full_rank(nparr_fill), count_full_rank(nparr_fill_res)))
        update_mask = np.isnan(nparr_fill)
        nparr_fill[update_mask] = nparr_fill_res[update_mask]

        # ------- forward filling -------
        good_data_mask = (np.sum(np.logical_not(np.isnan(nparr_fill)), axis=2) >= 8)

        # undefined indices are set to negative
        first_good_idx = np.zeros(good_data_mask.shape[1]) - 1 # set to -1

        for i1 in np.arange(good_data_mask.shape[0]):
            mask = first_good_idx < 0
            first_good_idx[mask] = (good_data_mask[i1][mask] * 2 - 1) * (i1 + 1) - 1

        # filling
        nparr_fill_res = nparr_fill.copy()

        for starting_idx in np.arange(9-1):  # 0 -> 7
            mask = first_good_idx == starting_idx

            if (show_progress):
                progress_bar = utl.TimedProgressBar(8 - starting_idx).update_msg("{0} forward pass, total {1}".format(starting_idx, np.count_nonzero(mask))).update()

            tcc_model_gru.step_reset()
            for time in np.arange(starting_idx, 8):
                nparr_fill_res[time + 1, mask] = tcc_model_gru.step_forward(nparr_fill[time, mask])

                if (show_progress):
                    progress_bar.inc_val().update()
            if (show_progress):
                print()

        # update
        if (show_progress):
            print("Full Rank Data: {0} -> {1}".format(count_full_rank(nparr_fill), count_full_rank(nparr_fill_res)))
        update_mask = np.isnan(nparr_fill)
        nparr_fill[update_mask] = nparr_fill_res[update_mask]

        if (show_progress):
            print("RESULT: Full Rank Data: {0}".format(count_full_rank(nparr_fill)))

        return nparr_fill

    def day_night_only_fill_helper(nparr_fill):
        """
        This help fill the day and night data for each sample
        If the sample does not have clue, then it will be randomly assigned. 
        """
        nparr_fill = nparr_fill.copy()
        day_night_idx = 3

        # creating template
        template_day_night = []
        for i0 in np.arange(nparr_fill.shape[0]):
            template_day_night.append(i0 % 2)
        template_night_day = np.array(template_day_night)
        template_day_night = 1 - template_night_day

        # fill
        for i1 in np.arange(nparr_fill.shape[1]):
            if (np.count_nonzero(np.isnan(nparr_fill[:, i1, day_night_idx])) < nparr_fill.shape[0]):
                # there are some data to interpolate
                for i0 in np.arange(nparr_fill.shape[0]):
                    if (not np.isnan(nparr_fill[i0, i1, day_night_idx])):
                        if (int(nparr_fill[i0, i1, day_night_idx]) == i0 % 2):
                            nparr_fill[:, i1, day_night_idx] = template_night_day
                        else:
                            nparr_fill[:, i1, day_night_idx] = template_day_night
                        break
            else:
                # no data avaliable -> randomize
                if (np.random.rand() > 0.5):
                    nparr_fill[:, i1, day_night_idx] = template_night_day
                else:
                    nparr_fill[:, i1, day_night_idx] = template_day_night


        return nparr_fill

    def mean_fill(nparr_fill):
        """
        Fill data using mean value of each column
        """
        nparr_fill = day_night_only_fill_helper(nparr_fill).copy()

        mean = np.nanmean(np.nanmean(nparr_fill, axis=1), axis=0)
        for i0 in np.arange(nparr_fill.shape[0]):
            for i1 in np.arange(nparr_fill.shape[1]):
                for i2 in np.arange(nparr_fill.shape[2]):
                    if (np.isnan(nparr_fill[i0, i1, i2])):
                        nparr_fill[i0, i1, i2] = mean[i2]

        return nparr_fill

    def daily_mean_fill(nparr_fill):
        """
        Fill data using mean value of each day and each column
        """
        nparr_fill = day_night_only_fill_helper(nparr_fill).copy()

        mean = np.nanmean(nparr_fill, axis=1)
        for i0 in np.arange(nparr_fill.shape[0]):
            for i1 in np.arange(nparr_fill.shape[1]):
                for i2 in np.arange(nparr_fill.shape[2]):
                    if (np.isnan(nparr_fill[i0, i1, i2])):
                        nparr_fill[i0, i1, i2] = mean[i0, i2]

        return nparr_fill

    def day_night_mean_fill(nparr_fill):
        """
        Fill data using day or night mean value of each column 
        """
        nparr_fill = day_night_only_fill_helper(nparr_fill).copy()

        day_night_idx = 3

        day_mean = np.nanmean(nparr_fill[nparr_fill[:, :, day_night_idx] > 0.5], axis=0)
        night_mean = np.nanmean(nparr_fill[nparr_fill[:, :, day_night_idx] < 0.5], axis=0)
        for i0 in np.arange(nparr_fill.shape[0]):
            for i1 in np.arange(nparr_fill.shape[1]):
                for i2 in np.arange(nparr_fill.shape[2]):
                    if (np.isnan(nparr_fill[i0, i1, i2])):
                        if (nparr_fill[i0, i1, day_night_idx] > 0.5):
                            nparr_fill[i0, i1, i2] = day_mean[i2]
                        else:
                            nparr_fill[i0, i1, i2] = night_mean[i2]

        return nparr_fill

    def linear_fill(nparr_fill):
        """
        Fill data using linear interpolation along time. 
        (mean value will be used if there no useful data)
        """
        def linear_fill_sample(sample, mean):
            if (np.count_nonzero(np.isnan(sample)) >= sample.shape[0]):
                # There is no data for the entire sample
                sample = np.ones(sample.shape) * mean
            else:
                # linear interpolation
                prev_idx = -1
                for curr_idx in np.arange(sample.shape[0]):
                    if (not np.isnan(sample[curr_idx])):
                        if (prev_idx != curr_idx):
                            # need some filling to do
                            if (prev_idx == -1):
                                # no begining data yet
                                sample[:curr_idx] = sample[curr_idx]
                            else:
                                # linear interpolation
                                sample[prev_idx:curr_idx] = np.polyval(
                                    np.polyfit([prev_idx, curr_idx],[sample[prev_idx], sample[curr_idx]], 1), 
                                    np.arange(prev_idx, curr_idx)
                                )

                        prev_idx = curr_idx
                if (prev_idx != sample.shape[0] - 1):
                    # need some filing at the end
                    sample[prev_idx:] = sample[prev_idx]

            return sample


        nparr_fill = day_night_only_fill_helper(nparr_fill).copy()

        day_night_idx = 3

        mean = np.nanmean(np.nanmean(nparr_fill, axis=1), axis=0) # for no data file. 

        for i1 in np.arange(nparr_fill.shape[1]):
            for i2 in np.arange(nparr_fill.shape[2]):
                nparr_fill[:, i1, i2] = linear_fill_sample(nparr_fill[:, i1, i2], mean[i2])

        return nparr_fill 

    def day_night_linear_fill(nparr_fill):
        """
        Fill data using linear interpolation along time. 
        (mean value will be used if there no useful data)
        """
        def linear_fill_sample(sample, mean):
            if (np.count_nonzero(np.isnan(sample)) >= sample.shape[0]):
                # There is no data for the entire sample
                sample = np.ones(sample.shape) * mean
            else:
                # linear interpolation
                prev_idx = -1
                for curr_idx in np.arange(sample.shape[0]):
                    if (not np.isnan(sample[curr_idx])):
                        if (prev_idx != curr_idx):
                            # need some filling to do
                            if (prev_idx == -1):
                                # no begining data yet
                                sample[:curr_idx] = sample[curr_idx]
                            else:
                                # linear interpolation
                                sample[prev_idx:curr_idx] = np.polyval(
                                    np.polyfit([prev_idx, curr_idx],[sample[prev_idx], sample[curr_idx]], 1), 
                                    np.arange(prev_idx, curr_idx)
                                )

                        prev_idx = curr_idx
                if (prev_idx != sample.shape[0] - 1):
                    # need some filing at the end
                    sample[prev_idx:] = sample[prev_idx]

            return sample


        nparr_fill = day_night_only_fill_helper(nparr_fill).copy()

        day_night_idx = 3

        day_mean = np.nanmean(nparr_fill[nparr_fill[:, :, day_night_idx] > 0.5], axis=0)
        night_mean = np.nanmean(nparr_fill[nparr_fill[:, :, day_night_idx] < 0.5], axis=0) 

        for i1 in np.arange(nparr_fill.shape[1]):
            for i2 in np.arange(nparr_fill.shape[2]):
                day_mask = nparr_fill[:, i1, day_night_idx] > 0.5
                night_mask = nparr_fill[:, i1, day_night_idx] < 0.5

                nparr_fill[:, i1, i2][day_mask] = linear_fill_sample(nparr_fill[:, i1, i2][day_mask], day_mean[i2])
                nparr_fill[:, i1, i2][night_mask] = linear_fill_sample(nparr_fill[:, i1, i2][night_mask], night_mean[i2])

        return nparr_fill 

    def cubic_spline_fill(nparr_fill):
        def cubic_spline_sample(sample, mean):
            timestamp = []
            avaliable_data = []
            for (time, data) in zip(np.arange(sample.shape[0]), sample):
                if (not np.isnan(data)):
                    timestamp.append(time)
                    avaliable_data.append(data)
            timestamp = np.array(timestamp)
            avaliable_data = np.array(avaliable_data)

            x = np.concatenate(([-1], timestamp, [9]))
            y = np.concatenate(([mean], avaliable_data, [mean]))
            if (len(x) > 3):
                return sp_interp.interp1d(x, y, kind="cubic")(np.arange(9))
            else:
                return sp_interp.interp1d(x, y, kind="quadratic")(np.arange(9))

        nparr_fill = nparr_fill.copy()
        mean = np.nanmean(np.nanmean(nparr_fill, axis=1), axis=0)
        for i1 in np.arange(nparr_fill.shape[1]):
            for i2 in np.arange(nparr_fill.shape[2]):
                nparr_fill[:, i1, i2] = cubic_spline_sample(nparr_fill[:, i1, i2], mean[i2])
        return nparr_fill

    def GRU_linear_hybrid_mux_fill(nparr_fill, GRU_threshold=4, tcc_model_gru_fill=None, tcc_model_gru_RRR_fill=None):
        """
        This function will calculate the GRU fill and linear fill of both function, 
            then determine which one will be in the final result.

        GRU_threshold: int, default as 2
            if there are n consecutive blanks in the sample
            where n > GRU_threshold, GRU filled sample will be chosen to fill the 
            final result
        """

        def GRU_linear_hybrid_fill_sample(sample, GRU_sample, linear_sample):
            x = sample

            # making a mask of fill types
            #   0 -> current is not a blank
            #   # -> sum # consecutive blanks before, after the idx and self
            x_mask = np.isnan(x).astype('int64')

            # always points to the index of blank starts - 1
            idx_start = -1
            for idx in np.arange(x_mask.shape[0]):
                if (x_mask[idx] == 0): # no blank at current idx
                    if (idx - idx_start > 2): # filling needed in between
                        x_mask[idx_start+1:idx] = idx - idx_start - 1
                    idx_start = idx
            idx += 1
            if (idx - idx_start > 2): # filling needed in between
                x_mask[idx_start+1:] = idx - idx_start - 1

            # filling
            for idx in np.arange(x.shape[0]):
                if (0 < x_mask[idx] and x_mask[idx] < GRU_threshold):
                    x[idx] = linear_sample[idx]
                elif (x_mask[idx] >= GRU_threshold):
                    x[idx] = GRU_sample[idx]

            return x


        if (tcc_model_gru_fill is None): 
            tcc_model_gru_fill = tcc_model_gru

        if (tcc_model_gru_RRR_fill is None): 
            tcc_model_gru_RRR_fill = tcc_model_gru_RRR

        nparr_fill_GRU = GRU_fill(nparr_fill.copy(), tcc_model_gru_fill, tcc_model_gru_RRR_fill, show_progress=False)
        nparr_fill_linear = linear_fill(nparr_fill.copy())

        for i1 in np.arange(nparr_fill.shape[1]):
            for i2 in np.arange(nparr_fill.shape[2]):
                nparr_fill[:, i1, i2] = GRU_linear_hybrid_fill_sample(
                    nparr_fill[:, i1, i2],
                    nparr_fill_GRU[:, i1, i2],
                    nparr_fill_linear[:, i1, i2]
                )

        return nparr_fill

    def GRU_linear_hybrid_fill(nparr_fill, GRU_threshold=4, tcc_model_gru_fill=None, tcc_model_gru_RRR_fill=None):
        """
        This function will will use GRU fill based on the linear filled sample

        GRU_threshold: int, default as 2
            if there are n consecutive blanks in the sample
            where n > GRU_threshold, GRU filled sample will be chosen to fill the 
            final result
        """

        def GRU_linear_hybrid_fill_sample(sample, linear_sample):
            x = sample

            # making a mask of fill types
            #   0 -> current is not a blank
            #   # -> sum # consecutive blanks before, after the idx and self
            x_mask = np.isnan(x).astype('int64')

            # always points to the index of blank starts - 1
            idx_start = -1
            for idx in np.arange(x_mask.shape[0]):
                if (x_mask[idx] == 0): # no blank at current idx
                    if (idx - idx_start > 2): # filling needed in between
                        x_mask[idx_start+1:idx] = idx - idx_start - 1
                    idx_start = idx
            idx += 1
            if (idx - idx_start > 2): # filling needed in between
                x_mask[idx_start+1:] = idx - idx_start - 1

            # filling
            for idx in np.arange(x.shape[0]):
                if (0 < x_mask[idx] and x_mask[idx] < GRU_threshold):
                    x[idx] = linear_sample[idx]

            return x


        if (tcc_model_gru_fill is None): 
            tcc_model_gru_fill = tcc_model_gru

        if (tcc_model_gru_RRR_fill is None): 
            tcc_model_gru_RRR_fill = tcc_model_gru_RRR

        nparr_fill_linear = linear_fill(nparr_fill.copy())

        for i1 in np.arange(nparr_fill.shape[1]):
            for i2 in np.arange(nparr_fill.shape[2]):
                nparr_fill[:, i1, i2] = GRU_linear_hybrid_fill_sample(
                    nparr_fill[:, i1, i2],
                    nparr_fill_linear[:, i1, i2]
                )

        nparr_fill = GRU_fill(nparr_fill.copy(), tcc_model_gru_fill, tcc_model_gru_RRR_fill, show_progress=False)

        return nparr_fill

    ## Bench Mark Models

    ### Model Definition

    #### GRU

    class MLTCC_Model_GRU_Normalized_TCC:
        def __init__(self):
            self.NMR = utl.Normalizer()

            self.input_size = 8
            self.output_size = 1
            self.batch_size = 10000
            self.step_size = 1e-3

            h_gru = 50
            h1 = 20
            h2 = 40
            h3 = 20

            self.gru = nn.GRU(self.input_size, h_gru, 2, batch_first=False)   # (seq, batch, feature)
            self.fc1 = nn.Linear(h_gru, h1)
            self.fc2 = nn.Linear(h1, h2)
            self.fc3 = nn.Linear(h2, h3)
            self.fc4 = nn.Linear(h3, self.output_size)

            variable_list = [
                    self.fc1.weight, self.fc1.bias,
                    self.fc2.weight, self.fc2.bias,
                    self.fc3.weight, self.fc3.bias,
                    self.fc4.weight, self.fc4.bias
                ]
            for all_weights in self.gru.all_weights:
                for weights in all_weights:
                    variable_list.append(weights)


            self.optim = torch.optim.Adam(
                variable_list,
                lr=self.step_size
            )

        def foward(self, X):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
            y := [TCC],                  (sample_size, 1) like tensor
            """
    #         seq_X_result = []
    #         for seq_X in X:
    #             seq_X_result.append(torch.relu(self.fc1(seq_X.float())))
    #         seq_X_result = tsr(seq_X_result)
            # normalize
            newX = []
            for idx in np.arange(X.shape[0]):
                newX.append(self.NMR.normalize_input(X[idx]).detach().numpy())
            X = torch.tensor(np.array(newX))

            X = X.float()

            X_gru_outputs, hn = self.gru(X)
            X = torch.relu(X_gru_outputs[-1]) # use last output

            X = torch.relu(self.fc1(X))
            X = torch.relu(self.fc2(X))
            X = torch.relu(self.fc3(X))
            X = self.fc4(X)

            return self.NMR.denormalize_output(X)

        def loss(self, y, y_hat):
            """
            y := [TCC],                  (sample_size, 1) like tensor
            y_hat := [TCC],              (sample_size, 1) like tensor
            """
            return F.mse_loss(y.float(), y_hat.float())

        def train(self, X, y_hat, max_epoch):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """
            sample_count = X.shape[1]

            epoch_num = []
            losses = []

            for i in np.arange(max_epoch):
                for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                    X_batch = torch.tensor(X[:, idx_batch:idx_batch + self.batch_size, :])
                    y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                    y_batch = self.foward(X_batch)
                    loss = self.loss(y_batch.t(), y_hat_batch.t())

                    self.optim.zero_grad()
                    loss.backward()
                    self.optim.step()

                idx_rand = np.random.randint(0, sample_count, [self.batch_size])
                X_batch = torch.tensor(X[:, idx_rand, :]).float()
                y_hat_batch = torch.tensor(y_hat[idx_rand, :]).float()
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)

                epoch_num.append(i)
                losses.append(loss.item())

            return (epoch_num, losses)


        def train_time_series(self, X, y_hat, max_epoch, max_epoch_per_time):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """
            progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")

            # setting normalizer
            # flatten X and y_hat
            X_flat = np.concatenate(X, axis=0)
            y_hat_flat = np.concatenate(y_hat, axis=0)

            self.NMR.set_mean_and_sd(torch.tensor(X_flat), torch.tensor(y_hat_flat))


            full_losses = np.array([])

            T_running = utl.get_runtime_marker()
            for i in np.arange(max_epoch):
                for time in np.arange(9):
                    (x_axis, losses) = self.train(X[:time+1, :, :], y_hat[time, :, :], max_epoch_per_time)
                    full_losses = np.concatenate((full_losses, losses))
                progress_bar.update(i + 1, "epoch {0}".format(i))
                
            progress_bar.update().end()

            return (np.arange(len(full_losses)) + 1, full_losses)

    #### FC

    class MLTCC_Model_FC_Normalized_TCC:
        def __init__(self):
            self.NMR = utl.Normalizer()

            self.input_size = 8
            self.output_size = 1
            self.batch_size = 10000
            self.step_size = 1e-3

            h1 = 20
            h2 = 40
            h3 = 20

            W0_rand = 1/np.sqrt(self.input_size)
            W0_init = np.random.uniform(-W0_rand, W0_rand, [h1, self.input_size])

            b0_rand = 1/np.sqrt(1)
            b0_init = np.random.uniform(-b0_rand, b0_rand, [h1, 1])

            W1_rand = 1/np.sqrt(h1)
            W1_init = np.random.uniform(-W1_rand, W1_rand, [h2, h1])

            b1_rand = 1/np.sqrt(1)
            b1_init = np.random.uniform(-b1_rand, b1_rand, [h2, 1])

            W2_rand = 1/np.sqrt(h2)
            W2_init = np.random.uniform(-W1_rand, W1_rand, [h3, h2])

            b2_rand = 1/np.sqrt(1)
            b2_init = np.random.uniform(-b1_rand, b1_rand, [h3, 1])

            W3_rand = 1/np.sqrt(h3)
            W3_init = np.random.uniform(-W1_rand, W1_rand, [self.output_size, h3])

            b3_rand = 1/np.sqrt(1)
            b3_init = np.random.uniform(-b1_rand, b1_rand, [self.output_size, 1])

            self.W0 = torch.tensor(W0_init, requires_grad=True)
            self.b0 = torch.tensor(b0_init, requires_grad=True)
            self.W1 = torch.tensor(W1_init, requires_grad=True)
            self.b1 = torch.tensor(b1_init, requires_grad=True)
            self.W2 = torch.tensor(W2_init, requires_grad=True)
            self.b2 = torch.tensor(b2_init, requires_grad=True)
            self.W3 = torch.tensor(W3_init, requires_grad=True)
            self.b3 = torch.tensor(b3_init, requires_grad=True)
    #         self.fc1 = nn.Linear(self.input_size, h1)
    #         self.fc2 = nn.Linear(h1, h2)
    #         self.fc3 = nn.Linear(h2, self.output_size)

            self.optim = torch.optim.Adam(
                [
                    self.W0, self.b0,
                    self.W1, self.b1,
                    self.W2, self.b2,
                    self.W3, self.b3
                ],
    #             [
    #                 self.fc1.weight, self.fc1.bias,
    #                 self.fc2.weight, self.fc2.bias,
    #                 self.fc3.weight, self.fc3.bias,
    #             ],
                lr=self.step_size
            )

        def foward(self, X):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
            y := [TCC],                  (sample_size, 1) like tensor
            """
            # W1 * (relu(W0 * X + b0)) + b1

    #         X_ = X.clone().type(torch.FloatTensor)

    #         X_ = F.relu(self.fc1(X_))
    #         X_ = F.relu(self.fc2(X_))
    #         X_ = self.fc3(X_)

    #         return X_
    #         X = torch.tanh(torch.matmul(self.W0, X.t()) + self.b0)

            X = self.NMR.normalize_input(X)

            X = torch.relu(torch.matmul(self.W0, X.t()) + self.b0)
            X = torch.relu(torch.matmul(self.W1, X) + self.b1)
            X = torch.relu(torch.matmul(self.W2, X) + self.b2)
            X = torch.matmul(self.W3, X) + self.b3
            return self.NMR.denormalize_output(X.t())

        def loss(self, y, y_hat):
            """
            y := [TCC],                  (sample_size, 1) like tensor
            y_hat := [TCC],              (sample_size, 1) like tensor
            """
            return F.mse_loss(y, y_hat)

        def train(self, X, y_hat, max_epoch):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """
            progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")

            self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))

            sample_count = X.shape[0]

            epoch_num = []
            losses = []

            for i in np.arange(max_epoch):
                for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                    X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                    y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                    y_batch = self.foward(X_batch)
                    loss = self.loss(y_batch, y_hat_batch)

                    self.optim.zero_grad()
                    loss.backward()
                    self.optim.step()

                idx_rand = np.random.randint(0, sample_count, [self.batch_size])
                X_batch = torch.tensor(X[idx_rand, :])
                y_hat_batch = torch.tensor(y_hat[idx_rand, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)

                epoch_num.append(i)
                losses.append(loss.item())

                progress_bar.update(i + 1, "epoch {0}".format(i))
                
            progress_bar.update().end()

            return (epoch_num, losses)

    #### Linear

    class MLTCC_Model_Linear_Normalized_TCC:
        def __init__(self):
            self.NMR = utl.Normalizer()

            self.input_size = 8
            self.output_size = 1
            self.batch_size = 50000
            self.step_size = 1e-3

            W0_rand = 1/np.sqrt(self.input_size)
            W0_init = np.random.uniform(-W0_rand, W0_rand, [self.output_size, self.input_size])

            b0_rand = 1/np.sqrt(1)
            b0_init = np.random.uniform(-b0_rand, b0_rand, [self.output_size, 1])

            self.W0 = torch.tensor(W0_init, requires_grad=True)
            self.b0 = torch.tensor(b0_init, requires_grad=True)

            self.optim = torch.optim.Adam(
                [
                    self.W0, self.b0
                ],
                lr=self.step_size
            )

        def foward(self, X):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
            y := [TCC],                  (sample_size, 1) like tensor
            """

            X = self.NMR.normalize_input(X)

            X = torch.matmul(self.W0, X.t()) + self.b0

            return self.NMR.denormalize_output(X.t())

        def loss(self, y, y_hat):
            """
            y := [TCC],                  (sample_size, 1) like tensor
            y_hat := [TCC],              (sample_size, 1) like tensor
            """
            return F.mse_loss(y, y_hat)

        def train(self, X, y_hat, max_epoch):
            """
            X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
            y_hat := [TCC],              (sample_size, 1) like matrix
            """
            progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")

            self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))

            sample_count = X.shape[0]

            epoch_num = []
            losses = []

            for i in np.arange(max_epoch):
                for idx_batch in np.arange(int(sample_count / self.batch_size))* self.batch_size:
                    X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                    y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                    y_batch = self.foward(X_batch)
                    loss = self.loss(y_batch, y_hat_batch)

                    self.optim.zero_grad()
                    loss.backward()
                    self.optim.step()

                idx_rand = np.random.randint(0, sample_count, [self.batch_size])
                X_batch = torch.tensor(X[idx_rand, :])
                y_hat_batch = torch.tensor(y_hat[idx_rand, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)

                epoch_num.append(i)
                losses.append(loss.item())

                progress_bar.update(i + 1, "epoch {0}".format(i))
                
            progress_bar.update().end()

            return (epoch_num, losses)


    ### Bench Mark Model Training

    # # GRU
    # tcc_model_gru_TCC = MLTCC_Model_GRU_Normalized_TCC()

    # (x_axis, losses) = tcc_model_gru_TCC.train_time_series(X_train, y_hat_train[:, :, -1:], 50, 2)

    # # FC
    # X_train_FC = flatten_time_axis(X_train)
    # y_hat_train_FC = flatten_time_axis(y_hat_train[:, :, -1:])
    # X_test_FC = flatten_time_axis(X_test)
    # y_hat_test_FC = flatten_time_axis(y_hat_test[:, :, -1:])

    # tcc_model_FC_TCC = MLTCC_Model_FC_Normalized_TCC()

    # (x_axis, losses) = tcc_model_FC_TCC.train(X_train_FC, y_hat_train_FC, 2000)

    # # Linear
    # X_train_Linear = flatten_time_axis(X_train)
    # y_hat_train_Linear = flatten_time_axis(y_hat_train[:, :, -1:])
    # X_test_Linear = flatten_time_axis(X_test)
    # y_hat_test_Linear = flatten_time_axis(y_hat_test[:, :, -1:])

    # tcc_model_Linear_TCC = MLTCC_Model_Linear_Normalized_TCC()

    # (x_axis, losses) = tcc_model_Linear_TCC.train(X_train_Linear, y_hat_train_Linear, 500)

    ## MSE, MAE Bench Mark Functions; Data Masking Functions

    def MSE(x_fill, x, masked):
        
        x_mean_arr = np.mean(x, axis=(0, 1))
        x_std_arr = np.std(x, axis=(0, 1))

        for idx in np.arange(x_std_arr.shape[0]):
            if (np.abs(x_std_arr[idx]) < 1e-7):
                x_std_arr[idx] = 1
        x_norm = (x - x_mean_arr) / x_std_arr
        x_fill_norm = (x_fill - x_mean_arr) / x_std_arr
        
        ret_val = []
        
        for idx in np.arange(x_fill.shape[2]):
            if (np.count_nonzero(np.isnan(masked[:, :, idx])) == 0):
                # No data are masked
                ret_val.append(0)
            else:
                if USING_STANDARD_DIVIATION:
                    ret_val.append(np.mean(np.power(x_fill_norm[:, :, idx] - x_norm[:, :, idx], 2)))
                else:
                    ret_val.append(np.mean(np.power(x_fill[:, :, idx] - x[:, :, idx], 2)))
        
        return np.array(ret_val)

    def MAE(x_fill, x, masked):
        x_mean_arr = np.mean(x, axis=(0, 1))
        x_std_arr = np.std(x, axis=(0, 1))

        for idx in np.arange(x_std_arr.shape[0]):
            if (np.abs(x_std_arr[idx]) < 1e-7):
                x_std_arr[idx] = 1
        x_norm = (x - x_mean_arr) / x_std_arr
        x_fill_norm = (x_fill - x_mean_arr) / x_std_arr
        
        ret_val = []
        
        for idx in np.arange(x_fill.shape[2]):
            if (np.count_nonzero(np.isnan(masked[:, :, idx])) == 0):
                # No data are masked
                ret_val.append(0)
            else:
                if USING_STANDARD_DIVIATION:
                    ret_val.append(np.mean(np.abs(x_fill_norm[:, :, idx] - x_norm[:, :, idx])))
                else:
                    ret_val.append(np.mean(np.abs(x_fill[:, :, idx] - x[:, :, idx])))
        
        return np.array(ret_val)


    def mask_data(a, percent_missing_data):
        a = a.copy()
        a.ravel()[np.random.choice(a.size, int(percent_missing_data * a.size), replace=False)]= np.nan
        return a

    def mask_data_column(a, percent_missing_data):
        """
        This function will perserve at least one timestamp per sample
        """
        a = a.copy()

        # save at least 1 column
        preserve_time_idx = np.random.randint(a.shape[0], size=a.shape[1])
        perserve_time_data = []
        for i1 in np.arange(a.shape[1]):
            perserve_time_data.append(a[preserve_time_idx[i1], i1, :])

        a = mask_data(a, percent_missing_data)

        for i1 in np.arange(a.shape[1]):
            a[preserve_time_idx[i1], i1, :] = perserve_time_data[i1]

        return a

    def mask_data_column_realistic(a, percent_missing_data):
        """
        This function will perserve at least one timestamp per sample
        The masked data will be contiguous chuck
        """
        a = a.copy()

        # progress = utl.TimedProgressBar(a.shape[1]).update_msg("Masking Data; percent masking: {0}".format(percent_missing_data)).update()

        for i1 in np.arange(a.shape[1]):
            wipe_data_count = np.minimum(np.count_nonzero(np.random.random(9) <= percent_missing_data), 8) # keep at least 1 column

            if (wipe_data_count > 0):
                # we need to wipe some data
                start_idx = np.random.randint(a.shape[0] - wipe_data_count + 1) # inclusive
                end_idx = start_idx + wipe_data_count                           # exclusive
                a[start_idx:end_idx, i1, :] = np.nan

            # progress.inc_val().update()

        # progress.update().end()

        return a

    ## Random Masking

    percent_list = np.arange(0, 1, 0.1) # from 0 to 0.9 with an increment of 0.1
    fill_MSE = {}
    fill_MAE = {}

    X_test_filled_overall = {}

    fill_MSE["PercentMissing"] = percent_list
    fill_MAE["PercentMissing"] = percent_list

    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Generate Data").update()
    masked_data = []
    X_test_filled_overall["Masked"] = []
    for percent_missing in percent_list:
        masked_data.append(mask_data_column(X_test, percent_missing)) 
        X_test_filled_overall["Masked"].append(masked_data[-1])
        progress_bar.inc_val().update()
    print()

    fill_MSE["GRU"] = []
    fill_MAE["GRU"] = []
    X_test_filled_overall["GRU"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("GRU Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = GRU_fill(masked_data_sample.copy(), tcc_model_gru, tcc_model_gru_RRR, show_progress=False)
        X_test_filled_overall["GRU"].append(X_test_filled)

        fill_MSE["GRU"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["GRU"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()


    fill_MSE["Mean"] = []
    fill_MAE["Mean"] = []
    X_test_filled_overall["Mean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = mean_fill(masked_data_sample.copy())
        X_test_filled_overall["Mean"].append(X_test_filled)

        fill_MSE["Mean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Mean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()


    fill_MSE["DailyMean"] = []
    fill_MAE["DailyMean"] = []
    X_test_filled_overall["DailyMean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Daily Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = daily_mean_fill(masked_data_sample.copy())
        X_test_filled_overall["DailyMean"].append(X_test_filled)

        fill_MSE["DailyMean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DailyMean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["DayNightMean"] = []
    fill_MAE["DayNightMean"] = []
    X_test_filled_overall["DayNightMean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Day Night Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = day_night_mean_fill(masked_data_sample.copy())
        X_test_filled_overall["DayNightMean"].append(X_test_filled)

        fill_MSE["DayNightMean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DayNightMean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["Linear"] = []
    fill_MAE["Linear"] = []
    X_test_filled_overall["Linear"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Linear Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = linear_fill(masked_data_sample.copy())
        X_test_filled_overall["Linear"].append(X_test_filled)

        fill_MSE["Linear"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Linear"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["DayNightLinear"] = []
    fill_MAE["DayNightLinear"] = []
    X_test_filled_overall["DayNightLinear"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Day Night Linear Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = day_night_linear_fill(masked_data_sample.copy())
        X_test_filled_overall["DayNightLinear"].append(X_test_filled)

        fill_MSE["DayNightLinear"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DayNightLinear"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    # fill_MSE["Cubic"] = []
    # fill_MAE["Cubic"] = []
    # X_test_filled_overall["Cubic"] = []
    # progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Cubic Spline Fill").update()
    # for masked_data_sample in masked_data:
    #     X_test_filled = cubic_spline_fill(masked_data_sample.copy())
    #     X_test_filled_overall["Cubic"].append(X_test_filled)

    #     fill_MSE["Cubic"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
    #     fill_MAE["Cubic"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

    #     progress_bar.inc_val().update()

    # progress_bar.update().end()

    # fill_MSE["HybridMux"] = []
    # fill_MAE["HybridMux"] = []
    # X_test_filled_overall["HybridMux"] = []
    # progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Hybrid Mux Fill").update()
    # for masked_data_sample in masked_data:
    #     X_test_filled = GRU_linear_hybrid_mux_fill(masked_data_sample.copy(), 4, tcc_model_gru, tcc_model_gru_RRR)
    #     X_test_filled_overall["HybridMux"].append(X_test_filled)

    #     fill_MSE["HybridMux"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
    #     fill_MAE["HybridMux"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

    #     progress_bar.inc_val().update()

    # progress_bar.update().end()

    fill_MSE["Hybrid"] = []
    fill_MAE["Hybrid"] = []
    X_test_filled_overall["Hybrid"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Hybrid Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = GRU_linear_hybrid_fill(masked_data_sample.copy(), 4, tcc_model_gru, tcc_model_gru_RRR)
        X_test_filled_overall["Hybrid"].append(X_test_filled)

        fill_MSE["Hybrid"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Hybrid"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    save_all_data(RANDOM_DATA_FOLDER_ARCHIVE, utl.get_current_day_time_string(), fill_MSE, fill_MAE)



    ## Realistic Masking
    # The data will be mask in contiguous chuck

    percent_list = np.arange(0, 1, 0.1) # from 0 to 0.9 with an increment of 0.1
    fill_MSE = {}
    fill_MAE = {}
    X_test_filled_overall = {}

    fill_MSE["PercentMissing"] = percent_list
    fill_MAE["PercentMissing"] = percent_list

    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Generate Data").update()
    masked_data = []
    X_test_filled_overall["Masked"] = []
    for percent_missing in percent_list:
        masked_data.append(mask_data_column_realistic(X_test, percent_missing)) 
        X_test_filled_overall["Masked"].append(masked_data[-1])
        progress_bar.inc_val().update()
    print()

    fill_MSE["GRU"] = []
    fill_MAE["GRU"] = []
    X_test_filled_overall["GRU"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("GRU Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = GRU_fill(masked_data_sample.copy(), tcc_model_gru, tcc_model_gru_RRR, show_progress=False)
        X_test_filled_overall["GRU"].append(X_test_filled)

        fill_MSE["GRU"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["GRU"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["Mean"] = []
    fill_MAE["Mean"] = []
    X_test_filled_overall["Mean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = mean_fill(masked_data_sample.copy())
        X_test_filled_overall["Mean"].append(X_test_filled)

        fill_MSE["Mean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Mean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["DailyMean"] = []
    fill_MAE["DailyMean"] = []
    X_test_filled_overall["DailyMean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Daily Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = daily_mean_fill(masked_data_sample.copy())
        X_test_filled_overall["DailyMean"].append(X_test_filled)

        fill_MSE["DailyMean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DailyMean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["DayNightMean"] = []
    fill_MAE["DayNightMean"] = []
    X_test_filled_overall["DayNightMean"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Day Night Mean Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = day_night_mean_fill(masked_data_sample.copy())
        X_test_filled_overall["DayNightMean"].append(X_test_filled)

        fill_MSE["DayNightMean"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DayNightMean"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["Linear"] = []
    fill_MAE["Linear"] = []
    X_test_filled_overall["Linear"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Linear Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = linear_fill(masked_data_sample.copy())
        X_test_filled_overall["Linear"].append(X_test_filled)

        fill_MSE["Linear"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Linear"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    fill_MSE["DayNightLinear"] = []
    fill_MAE["DayNightLinear"] = []
    X_test_filled_overall["DayNightLinear"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Day Night Linear Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = day_night_linear_fill(masked_data_sample.copy())
        X_test_filled_overall["DayNightLinear"].append(X_test_filled)

        fill_MSE["DayNightLinear"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["DayNightLinear"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    # fill_MSE["Cubic"] = []
    # fill_MAE["Cubic"] = []
    # X_test_filled_overall["Cubic"] = []
    # progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Cubic Spline Fill").update()
    # for masked_data_sample in masked_data:
    #     X_test_filled = cubic_spline_fill(masked_data_sample.copy())
    #     X_test_filled_overall["Cubic"].append(X_test_filled)

    #     fill_MSE["Cubic"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
    #     fill_MAE["Cubic"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

    #     progress_bar.inc_val().update()

    # progress_bar.update().end()


    # fill_MSE["HybridMux"] = []
    # fill_MAE["HybridMux"] = []
    # X_test_filled_overall["HybridMux"] = []
    # progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Hybrid Mux Fill").update()
    # for masked_data_sample in masked_data:
    #     X_test_filled = GRU_linear_hybrid_mux_fill(masked_data_sample.copy(), 4, tcc_model_gru, tcc_model_gru_RRR)
    #     X_test_filled_overall["HybridMux"].append(X_test_filled)

    #     fill_MSE["HybridMux"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
    #     fill_MAE["HybridMux"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

    #     progress_bar.inc_val().update()

    # progress_bar.update().end()

    fill_MSE["Hybrid"] = []
    fill_MAE["Hybrid"] = []
    X_test_filled_overall["Hybrid"] = []
    progress_bar = utl.TimedProgressBar(percent_list.shape[0]).update_msg("Hybrid Fill").update()
    for masked_data_sample in masked_data:
        X_test_filled = GRU_linear_hybrid_fill(masked_data_sample.copy(), 4, tcc_model_gru, tcc_model_gru_RRR)
        X_test_filled_overall["Hybrid"].append(X_test_filled)

        fill_MSE["Hybrid"].append(MSE(X_test_filled, X_test, masked_data_sample.copy()))
        fill_MAE["Hybrid"].append(MAE(X_test_filled, X_test, masked_data_sample.copy()))

        progress_bar.inc_val().update()

    progress_bar.update().end()

    save_all_data(REALISTIC_DATA_OUTPUT_FOLDER_PATH_ARCHIVE, utl.get_current_day_time_string(), fill_MSE, fill_MAE)

In [ ]:
for i in np.arange(100):
    T = utl.get_runtime_marker()
    work()
    print("{0}; {1}".format(i + 1, utl.format_time_s_2_hms(utl.get_runtime_in_second(T))))
    print()